In [3]:
import os
#os.chdir("../")
%pwd

'd:\\Projects\\Learning\\Wine-Quality-Prediction'

In [4]:
from pathlib import Path
from dataclasses import dataclass

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from wqpproject.constants import *
from wqpproject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir =  config.root_dir,
            train_data_path =  config.train_data_path,
            test_data_path =  config.test_data_path,
            model_name = config.model_name,
            alpha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )

        return model_trainer_config

In [9]:
import pandas as pd
import os
from wqpproject import logger
from sklearn.linear_model import ElasticNet
import joblib

class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        train_x = train_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]

        logger.info("Model training has started")
        model = ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        model.fit(train_x, train_y)
        joblib.dump(model, os.path.join(self.config.root_dir,self.config.model_name))
        logger.info("Model training is completed")

In [10]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e

[2023-09-10 18:44:57,697: INFO: common: yaml file : config\config.yaml loaded successfully]
[2023-09-10 18:44:57,701: INFO: common: yaml file : params.yaml loaded successfully]
[2023-09-10 18:44:57,706: INFO: common: yaml file : schema.yaml loaded successfully]
[2023-09-10 18:44:57,709: INFO: common: Created directory at : artifacts]
[2023-09-10 18:44:57,710: INFO: common: Created directory at : artifacts/model_trainer]
[2023-09-10 18:44:57,721: INFO: 1042936655: Model training has started]
[2023-09-10 18:44:57,732: INFO: 1042936655: Model training is completed]
